<a href="https://colab.research.google.com/github/BrotherKim/AI599/blob/master/Final_train_binary_prison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KAIST AI599 법률인공지능

In [1]:
# train variables
TARGET = 'prison'
TARGET_SET = 'avail' # all, avail, nonavail
TARGET_START = 0 # 0 if you want zero values, INCLUDE THST VALUE
TARGET_END = 36 # INCLUDE THIS VALUE
TARGET_APPROX = 0 # '2'개월 또는 '1000000'원
TARGET_OPT = 'binary' # class, binary
TRAIN_COLUMN = 'fy' # raw, facts, statutes, yh, fy
MODEL_DIR = '/content/drive/MyDrive/AI599/models'

## 과제 개요

이름: 강여진, 김동규, 김학성, 김형, 여인경

개요
- 컬럼은 한글로 사용
- model은 영대문자 사용
- dataset은 영소문자, 숫자 사용


데이터 준비
- precedent는 주문을 형량으로 바꾼 것(이하 "형"), 양형의 사유(이하 "양"), 그리고 나머지(이하 "나")로 이루어져 있음. 양형의 사유와 나머지를 포함한 것을 "양나"라고 칭함.
- "형"을 벌금만 있는 경우와 징역만 있는 경우, 그리고 집행유예가 있는 징역의 경우를 추출. 각각 "벌", "징", "집"이라 칭함.
- 양형의 사유가 없는 집합에서 "벌"-"나", "징"-"나", "집"-"나" 데이터셋 생성. 이를 a, b, c라고 칭함
- 양형의 사유가 있는 집합에서 "벌"-"나", "징"-"나", "집"-"나", "벌"-"양나", "징"-"양나", "집"-"양나" 데이터셋 생성. 이를 d, e, f, g, h, i라고 칭함.
- a~i까지 데이터 생성 시, 해당하지 않는 라벨에 대해서도 예측하도록 함.


판결문에 대한 형량 예측 모델 학습
- 모델 A, B 준비. A에는 a로 "나"에 대한 "벌" 예측 모델 생성. B에는 b로 "나"에 대한 "징" 예측 모델 생성, C에는 c로 "나"에 대한 "집" 예측 모델 생성.


예측
- A에 d를 eval시켜 "나"에 해당하는 "벌" 예측. 이 결과쌍을 j라고 칭함.
- B에 e를 eval시켜 "나"에 해당하는 "징" 예측. 이 결과쌍을 k라고 칭함.
- C에 f를 eval시켜 "나"에 해당하는 "집" 예측. 이 결과쌍을 l라고 칭함.


양형 사유에 대한 형량 예측 모델 학습
- d, g의 "벌"의 delta 값을 추출. "양"-delta 데이터셋 생성. 이하 m.
- e, h의 "징"의 delta 값을 추출. "징"-delta 데이터셋 생성. 이하 n.
- f, i의 "집"의 delta 값을 추출. "집"-delta 데이터셋 생성. 이하 o.
- 모델 C, D, E 준비. m, n, o에 대해 학습.

## Google drive connect.
connect google drive storage for saving checkpoints.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !rm -rf /content/drive/MyDrive/AI599/models
# !mkdir -p /content/drive/MyDrive/AI599/models

In [4]:
# # save models
# torch.save(models['prison'], '/content/drive/MyDrive/AI599/models//prison_model.pt')
# #torch.save(money_model, '/content/drive/MyDrive/AI599/models/money_model.pt')
# #torch.save(yooye_model, '/content/drive/MyDrive/AI599/models/yooye_model.pt')

## lbox_open data 전처리.
lbox_open 데이터를 가져와 형태를 분석합니다.

In [5]:
!pip install -q datasets==2.4.0

     |████████████████████████████████| 365 kB 6.6 MB/s 
     |████████████████████████████████| 132 kB 69.7 MB/s 
     |████████████████████████████████| 212 kB 68.1 MB/s 
     |████████████████████████████████| 95 kB 5.3 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 127 kB 80.4 MB/s 
     |████████████████████████████████| 131 kB 75.3 MB/s 


In [6]:
import datasets

dataset_card = "lbox/lbox_open"
task = "precedent_corpus"
data = datasets.load_dataset(dataset_card, task)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset lbox_open downloaded and prepared to /root/.cache/huggingface/datasets/lbox___lbox_open/precedent_corpus/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

데이터를 개별 확인해 보면, 아래와 같은 형태를 띄고 있는 것을 볼 수 있습니다.

In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'precedent'],
        num_rows: 150000
    })
})

In [8]:
data_t = data["train"]
data_t[40]

{'id': 40,
 'precedent': "주문\n1. 제1심판결을 취소한다.\n2. 원고의 청구를 기각한다\n3. 소송비용은 제1, 2심을 합하여 원고의 부담으로 한다.\n\n청구취지및항소취지\n1. 청구취지\n피고는 원고에게 1,500,000원 및 이에 대하여 2004. 7. 9.부터 다 갚는 날까지 연 20%의 비율에 의한 돈을 지급하라.\n2. 항소취지\n주문과 같다.\n\n이유\n1. 기초사실\n다음 각 사실은 당사자 사이에 다툼이 없거나 갑 제1 내지 3호증, 을 제1호증의 각 기재에 변론의 전취지를 종합하여 이를 인정할 수 있다.\n가. 원고는 2004. 4. 29. 주식회사 세기메디칼에 엑스레이필름대금 150만 원을 송금하려고 하였으나 거래 상대방에 대한 착오를 일으켜, 원고가 거래하던 한국외환은행 주식회사(이하 '외환은행'이라 한다)에 인터넷 뱅킹을 통하여 원고의 예금 150만 원을 피고의 나운동 지점에 개설된 소외인의 계좌로 이체를 의뢰하였고, 외환은행은 즉시 150만 원을 피고 은행에 송금하였으며, 피고는 동시에 소외인의 계좌에 150만 원의 입금기장을 하였다.\n나. 소외인은 ○○인쇄소를 경영하면서 2003. 8.경까지 원고와 거래하였던 자인데, 2001. 11. 24. 피고로부터 500만 원을 대출받고는 2003. 12. 25. 이후 원리금 5,153,821원을 연체하였다.\n다. 피고는 2004. 5. 6. 위 대출금채권을 자동채권으로 하고 소외인의 위 150만 원에 대한 예금반환채권을 수동채권으로 하여 상계처리하였다.\n2. 주장 및 판단\n가. 청구원인\n원고는 소외인과는 거래가 없었는데 피고에 개설된 그의 계좌를 주식회사 세기메디칼의 것으로 착오하는 바람에 소외인의 계좌로 150만 원을 입금한 것이므로, 피고가 소외인의 계좌에 150만 원이 입금된 것을 기화로 자신의 그에 대한 대출금채권과 동액 상당을 상계한 것은 법률상 원인 없이 부당이득한 것이므로, 피고는 원고에게 150만 원을 반환해야 할 의무가 있다.\n나. 판 단\n

## filtering lbox_open data
2019.6.25 이후에 발생한 음주운전 데이터를 추출합니다.

In [9]:
import pandas as pd
df = pd.DataFrame( (v for v in data['train']) )
# df = pd.Series( (v['precedent'] for v in data['train']) )

In [10]:
# 주문과 형량이 나타나는 사용 가능한 데이터 우선 추출
avail_df = df[ \
           df['precedent'].str.contains('법령의 적용') \
           & df['precedent'].str.contains('피고인을') \
           & df['precedent'].str.contains('처한다.') \
           & df['precedent'].str.contains('2020|2021') # 2019.6.25 개정 이후 사건만 다룸
           & df['precedent'].str.contains('도로교통법') \
           & df['precedent'].str.contains('제44조 제1항') 
           ]

print(len(avail_df))

3468


In [11]:
# 양형의 사유가 있는 것과 없는 것 분리
yesy = avail_df[avail_df['precedent'].str.contains('양형의 이유')]
noy = avail_df[~avail_df['precedent'].str.contains('양형의 이유')]
yesy
noy

,id,precedent
38,38,"주문\n피고인을 벌금 1,000만원에 처한다.\n피고인이 위 벌금을 납입하지 아니하..."
205,205,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일부터 2년간 위 형의 ..."
248,248,"주문\n피고인을 벌금 12,000,000원에 처한다.\n피고인이 위 벌금을 납입하지..."
757,757,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일로부터 2년간 위 형의..."
790,790,"주문\n피고인을 징역 1년에 처한다.\n다만, 이 판결 확정일로부터 2년간 위 형의..."
...,...,...
148388,148388,주문\n피고인을 징역 1년 2월에 처한다.\n다만 이 판결 확정일로부터 2년간 위 ...
148546,148546,"주문\n피고인을 징역 1년 6월에 처한다.\n다만, 이 판결 확정일부터 2년간 위 ..."
148775,148775,"주문\n피고인을 벌금 1,500만 원에 처한다.\n피고인이 위 벌금을 납입하지 아니..."
149213,149213,"주문\n피고인을 징역 2년에 처한다.\n다만, 이 판결 확정일부터 3년간 위 형의 ..."


In [12]:
# Precedent 쪼개기 위한 function set

import re

class Precedent:
    def __init__(self, rawPrecedent):
        self.raw = rawPrecedent
        self.__parse__()
        
    def __parse__(self):
        # 주문 - 범죄사실 - 법령의적용 - 양형의이유
        # ms - facts - statuts - yh
        # fy: facts + yh
        retval = re.search(r'주문[\n]*([^\n].+[^\n])[\n]+이유[\n]*([^\n].+[^\n])[\n]*', self.raw, re.S)
        self.ms = retval.group(1) # Main Sentence, 주문
        self.ms = self.ms.strip()

        self.etc = retval.group(2)

        self.yh = ''
        l = self.etc.split('양형의 이유')
        if 1 < len(l):
            self.etc = l[0].strip()
            self.yh = l[1].strip()

        self.statutes = ''
        l = self.etc.split('법령의 적용')
        if 1 < len(l):
            self.etc = l[0].strip()
            self.statutes = l[1].strip()

        self.facts = self.etc

        self.fy = f'{self.facts}\n{self.yh}'

        # self.etc = self.etc.replace('\n', ' ')
        # self.etc = self.etc.replace('에 대한 해당법조 및 형의 선택', '')
        # self.etc = self.etc.replace('에 대한 해당법조', '')
        # self.etc = self.etc.replace('형의 선택', '')
        # self.etc = self.etc.replace('피고인의 법정진술', '')
        # self.etc = self.etc.replace('[', '')
        # self.etc = self.etc.replace(']', '')
        # self.etc = self.etc.replace('1. ', '')
        # self.etc = self.etc.replace('2. ', '')
        # self.etc = self.etc.replace('3. ', '')
        # self.etc = self.etc.replace('4. ', '')
        # self.etc = self.etc.replace('범 죄 사 실', '')
        # self.etc = self.etc.replace('범죄전력', '')
        # self.etc = self.etc.replace('범죄사실', '')
        # self.etc = self.etc.replace('증거의 요지', '')
        # self.etc = self.etc.replace('법령의 적용', '')
        # self.etc = self.etc.replace('■', '')
        # self.etc = self.etc.strip()

        # self.ms = self.ms.replace('\n', ' ')
        # self.ms = self.ms.strip()

        self.raw = self.raw.strip()


        # self.etc = self.etc.replace('\n', ' ')
        # self.ms = self.ms.replace('\n', ' ')
        # self.raw = self.raw.replace('\n', ' ')
        # self.rsl = 'asf' # Reason of Sentence in law, 양형의 사유
        # self.statutes = 'df' # statutes, 법령

def getNumericInKorean(kStrNum):
    retval = 1
    retStr = kStrNum
    retStr = retStr.replace(',', '')
    if '만' in retStr:
        retval = retval * 10000
        retStr = retStr.replace('만', '')
    if '천' in retStr:
        retval = retval * 1000
        retStr = retStr.replace('천', '')
    retval = int(retStr) * retval
    return retval

def getMonthFromText(s):
    year  = re.findall(r'(\d+)년', s, re.S)
    y = 0 if len(year) == 0 else (int)(year[0]);
    month  = re.findall(r'(\d+)개?월', s, re.S)
    m = 0 if len(month) == 0 else (int)(month[0]);
    return y*12 + m

def generateDataSet(dataframe, max_cnt, classopt='class'):
    retval = pd.DataFrame({\
                           'id': pd.Series(dtype='int'), 'money': pd.Series(dtype='int'), 'prison': pd.Series(dtype='int'),'yooye': pd.Series(dtype='int'),\
                           'facts':[], 'statutes':[], 'yh':[], 'fy':[], 'raw':[]})
    cnts = {}
    for i, row in dataframe.iterrows():
        p = Precedent(row['precedent'])
        #print(f'p.ms: {p.ms}\n p.etc: {p.etc}')
        if '처한다' in p.ms and p.ms.startswith('피') and '및' not in p.ms:
            info = {}
            info['money'] = 0
            info['prison'] = 0
            info['yooye'] = 0
            # 주문\n피고인을 벌금 14,000,000(일천사백만)원에 처한다.\
            money  = re.findall(r'벌금\s+([^\s\(\)]+)\s*원', p.ms, re.S)
            if 0 < len(money): # 벌금형
                if classopt == 'class':
                    info['money'] = '0' if len(money) == 0 else getNumericInKorean(money[0])
                else:
                    info['money'] = '0' if len(money) == 0 else 1
            else: # 금고형, 집행유예 포함
                if '유예' in p.ms: # 집행유예 포함된 금고형.
                    msList = p.ms.split('처한다')
                    info['yooye'] = getMonthFromText(msList[1]) if classopt == 'class' else 1
                    info['prison'] = getMonthFromText(msList[0]) if classopt == 'class' else 1
                    # yy = 1
                    #prison = 1
                else: # 그냥 금고형
                    info['prison'] = getMonthFromText(p.ms) if classopt == 'class' else 1
                    #prison = 1

            if TARGET_START <= info[TARGET] and info[TARGET] <= TARGET_END:
            #    if prison in [0, 12, 18]:
                if info[TARGET] not in cnts:
                    cnts[info[TARGET]] = 0
                cnts[info[TARGET]] = cnts[info[TARGET]] + 1
                if cnts[info[TARGET]] > max_cnt:
                    continue
                retval.loc[i] = [i, [info['money']], [info['prison']], [info['yooye']], p.facts, p.statutes, p.yh, p.fy, p.raw]
            
            #retval.loc[i] = [i, [mn], [prison], [yy], p.etc, p.raw]  
    return retval

In [13]:
# 양형의 사유가 없는 집합에서 "벌"-"나", "징"-"나" 데이터셋 생성. 이를 a, b라고 칭함
dataset = {}
dataset['all'] = generateDataSet(avail_df, 2000, TARGET_OPT)
dataset['avail'] = generateDataSet(yesy, 1740, TARGET_OPT)
dataset['nonavail'] = generateDataSet(noy, 340, TARGET_OPT)

In [14]:
dataset['all'][TARGET].value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[1]    1830
[0]     673
Name: prison, dtype: int64

In [15]:
dataset['avail'][TARGET].value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[1]    1618
[0]     486
Name: prison, dtype: int64

In [16]:
dataset['nonavail'][TARGET].value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[1]    212
[0]    187
Name: prison, dtype: int64

In [17]:
# # hs's rule
# [1,2,3,4,5]
# [6,10]
# []
# []
# []

# # yk's rule
# [1 2 3]
# [4 5 6]
# [7 8 9] 8
# [10 11 12]
# [13 14 15] 14
# [16 17 18]

# # 1~5 / 6~8 / 9~12 / 13~15 / 16~18 / 19~22 / 24~29 / 30~36 /

## 전처리된 데이터로 학습.
모델이 학습할 수 있도록 데이터의 형태를 변경해 학습합니다.

In [18]:
!pip install --quiet transformers==4.22.0
!pip install --quiet sentencepiece==0.1.97
!pip install --quiet pytorch-lightning==1.7.6
!pip install --quiet sklearn

     |████████████████████████████████| 4.9 MB 7.9 MB/s 
     |████████████████████████████████| 6.6 MB 55.7 MB/s 
     |████████████████████████████████| 1.3 MB 6.7 MB/s 
     |████████████████████████████████| 707 kB 8.2 MB/s 
     |████████████████████████████████| 512 kB 64.8 MB/s 


In [19]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from functools import reduce
import random

import torch
import numpy as np
import transformers
import pytorch_lightning as pl
from transformers import AutoTokenizer, BertForSequenceClassification

import pandas as pd

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [20]:
class StatutesDataModule(pl.LightningDataModule):
    def __init__(self, column, trainset, statutes, tokenizer, data, batch_size=16, max_input_len=512):
        super().__init__()
        self.column = column
        self.trainset = trainset
        self.statutes = statutes
        self.tokenizer = tokenizer
        self.data = data
        self.batch_size = batch_size
        self.max_input_len = max_input_len 
    
    def setup(self, stage):
        pass
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.data['train'], batch_size=self.batch_size, shuffle=True, collate_fn=self._collate_fn)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.data['valid'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.data['test'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)
    
    def _collate_fn(self, batch):
        inputs = self.tokenizer([x[self.trainset] for x in batch], max_length=self.max_input_len, padding=True, truncation=True, return_tensors='pt')
        labels = [x[self.column] for x in batch]

        return inputs, labels

In [21]:
class StatutesClassifier(pl.LightningModule):
    def __init__(self, statutes, backbone, learning_rate=1e-5, mode='multi-class', approx_val=2):
        super().__init__()
        self.statutes = statutes
        self.backbone = backbone
        self.learning_rate = learning_rate
        self.mode = mode
        self.criterion = torch.nn.CrossEntropyLoss() if mode == 'multi-class' else torch.nn.BCEWithLogitsLoss()
        self.approx_val = approx_val
        #self.criterion = torch.nn.MSELoss()
        #self.criterion = torch.nn.BCEWithLogitsLoss()
    
    def forward(self, batch):
        inputs, labels = batch
        logits = self.backbone(**inputs).logits
        targets = torch.zeros_like(logits)
        for i, label in enumerate(labels):
            label_id = [(self.statutes.index(x) if x in self.statutes else 0) for x in label]
            targets[i, label_id] = 1 / len(label_id) if self.mode == 'multi-class' else 1
        loss = self.criterion(logits, targets)

        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward(batch)
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        return self._evaluation_step(batch) 
    
    def validation_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs)

    def test_step(self, batch, batch_idx):
        return self._evaluation_step(batch) 

    def test_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs, True)
   
    def _evaluation_step(self, batch):
        loss, logits = self.forward(batch)
        _, gts = batch
        if self.mode == 'multi-class':
            prs = np.array(self.statutes)[logits.argmax(-1).cpu()].tolist()
        else:
            prs = [np.array(self.statutes)[np.where(logit.sigmoid().cpu() > 0.5)[0]] for logit in logits]

        if isinstance(prs, list) == False:
            prs = [prs]
        
        return {'loss': loss.item(), 'gts': gts, 'prs': prs}

    def _evaluation_epoch_end(self, outputs, test=False):
        avg_loss = np.mean([x['loss'] for x in outputs])
        print(outputs)
        gts = reduce(lambda x,y: x + y, [x['gts'] for x in outputs], [])
        prs = reduce(lambda x,y: x + y, [x['prs'] for x in outputs], [])
        if self.mode == 'multi-class':
            #acc = sum([(pr in gt) for gt, pr in zip(gts, prs)]) / len(gts)
            acc = sum([abs(pr - gt[0] <= self.approx_val) for gt, pr in zip(gts, prs)]) / len(gts)
        else:
            acc = sum([(set(pr) == set(gt)) for gt, pr in zip(gts, prs)]) / len(gts)
        
        print('='*50)
        print(f'avg_loss: {avg_loss}')
        print(f'ACC: {acc}')
        
        #target_ids = random.sample(range(len(gts)), 2)
        target_ids = random.sample(range(len(prs)), 2)
        print(gts)
        print(prs)
        print(f'target_ids: {target_ids}')
        for target_id in target_ids:
            print(f'GT: {gts[target_id]}\t\t\tPR: {prs[target_id]}')
        
        if self.mode == 'multi-class' and test:
            #df = pd.DataFrame(columns=['idx', 'gt', 'pr', 'etc'])
            #df = pd.DataFrame({'idx': pd.Series(dtype='int'), 'gt':pd.Series(dtype='int'), 'pr':pd.Series(dtype='int'),'etc':[]})
            df = pd.DataFrame({'idx': pd.Series(dtype='int'), 'gt':pd.Series(dtype='int'), 'pr':pd.Series(dtype='int')})
            print(data)
            for i, (gt, pr) in enumerate(zip(gts, prs)):
                if pr not in gt:
                    # df = df.append(pd.DataFrame({'idx': [i],
                    #                              'gt': ', '.join((str)(gt)),
                    #                              'pr': pr, 
                    #                              'etc': data['test'][i]['etc']}),
                    #                 ignore_index=True)
                    df = df.append(pd.DataFrame({'idx': [i],
                                                 'gt': ', '.join((str)(gt)),
                                                 'pr': pr}),
                                    ignore_index=True)
            df.to_csv('GT_vs_PR.csv', index=False)
            

    def configure_optimizers(self):
        grouped_params = [{'params': list(filter(lambda p: p.requires_grad, self.parameters())), 'lr': self.learning_rate}]
        optimizer = torch.optim.AdamW(grouped_params, lr=self.learning_rate)

        return {'optimizer': optimizer}

In [22]:
from sklearn.model_selection import train_test_split
import datasets
import tensorflow as tf
from datasets import Dataset

def convertDFtoDS(df):
    return tf.data.Dataset.from_tensor_slices(dict(df))

backbone_card = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(backbone_card)


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [23]:
# # coded by cat
# cfdxf,er?"
# ]\w
# ws"

In [24]:
import gc

def execTrain(dataset, target, trainset):
    #init cuda
    gc.collect()
    torch.cuda.empty_cache()

    #divide train/valid/test set
    train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    train, valid = train_test_split(train, test_size=0.2, random_state=1)
    d = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

    # labels init
    labels = sorted(set.union(*map(set, dataset[target])))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, d, batch_size=16, max_input_len=512)

    # model
    backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(labels))
    model = StatutesClassifier(labels, backbone, learning_rate=1e-5, mode='multi-class', approx_val=TARGET_APPROX)

    # trainer
    n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    # train and eval
    trainer.fit(model, data_module)
    trainer.test(model, data_module)

    return model

def execEval(dataset, target, trainset, modelpath):
    #init cuda
    gc.collect()
    torch.cuda.empty_cache()

    #divide train/valid/test set
    train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    train, valid = train_test_split(train, test_size=0.2, random_state=1)
    d = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

    # labels init
    labels = sorted(set.union(*map(set, dataset[target])))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, d, batch_size=16, max_input_len=512)

    # model
    backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(labels))
    model = StatutesClassifier(labels, backbone, learning_rate=1e-5, mode='multi-class')
    model = torch.load(modelpath)

    # trainer
    n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    trainer.test(model, data_module)

## RUN

In [25]:
# check variables
print(TARGET)
print(TARGET_SET)
print(TARGET_START)
print(TARGET_END)
print(TARGET_APPROX)
print(TARGET_OPT)
print(TRAIN_COLUMN)

prison
avail
0
36
0
binary
fy


In [26]:
# 양형의 이유 있는 데이터 중 etc로 prison classification, MSE
model = execTrain(dataset[TARGET_SET], TARGET, TRAIN_COLUMN)
torch.save(model, f'{MODEL_DIR}/{TARGET}_{TARGET_SET}_{TRAIN_COLUMN}_{TARGET_START}_{TARGET_END}_model.pt')

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Sanity Checking: 0it [00:00, ?it/s]

[{'loss': 0.7020889520645142, 'gts': [[0], [0], [1], [1], [1], [1], [1], [0], [0], [0], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0]], 'prs': [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'loss': 0.6974797248840332, 'gts': [[0], [1], [1], [1], [1], [0], [1], [1], [1], [0], [1], [0], [1], [0], [1], [1], [1], [1], [1], [1], [0], [0], [1], [1], [1], [1], [0], [1], [1], [1], [1], [1]], 'prs': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]}]
avg_loss: 0.6997843384742737
ACC: 0.96875
[[0], [0], [1], [1], [1], [1], [1], [0], [0], [0], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [1], [1], [1], [1], [0], [1], [1], [1], [0], [1], [0], [1], [0], [1], [1], [1], [1], [1], [1], [0], [0], [1], [1], [1], [1], [0], [1], [1], [1], [1], [1]]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

[{'loss': 1.0657007694244385, 'gts': [[0], [0], [1], [1], [1], [1], [1], [0], [0], [0], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0]], 'prs': [0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'loss': 0.5977102518081665, 'gts': [[0], [1], [1], [1], [1], [0], [1], [1], [1], [0], [1], [0], [1], [0], [1], [1], [1], [1], [1], [1], [0], [0], [1], [1], [1], [1], [0], [1], [1], [1], [1], [1]], 'prs': [0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]}, {'loss': 0.32744547724723816, 'gts': [[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [1], [1], [0], [1], [1], [1]], 'prs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1]}, {'loss': 1.2387642860412598, 'gts': [[1], [1], [1], [1], [0], [0], [1], [0], [1], [1],

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'loss': 1.3729559183120728, 'gts': [[0], [1], [1], [1], [1], [1], [0], [1], [1], [0], [1], [0], [0], [0], [1], [0], [0], [1], [1], [0], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0]], 'prs': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'loss': 1.1247329711914062, 'gts': [[0], [1], [1], [1], [1], [1], [0], [1], [1], [1], [1], [0], [1], [1], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [1], [1], [1], [0], [1], [0]], 'prs': [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]}, {'loss': 0.6503938436508179, 'gts': [[1], [0], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1]], 'prs': [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]}, {'loss': 0.8444825410842896, 'gts': [[1], [1], [0], [0], [1], [0], [1], [0], [1], [0], 